# Priority Queues

In diesem Notebook werden wir zwei einfache Implementationen des ADTs *Priority Queue* anschauen. Diese Implementationen dienen nur zur Illustration des Prinzips, und zur Veranschaulichung, dass auch hier verschiedene Implementationen verschiedene Kompromisse machen. Für eine praktische Implementation werden wir später einen Heap verwenden.



## Zwei einfache Implementationen mittels Arrays

#### Implementation mit unsortiertem Array

Die erste Implementation verwendet ein einfaches, unsortiertes (dynamisches) Array. Bei dieser Implementation ist die ```insert``` Methode sehr effizient. Die Hauptarbeit passiert, wenn die Methode ```max``` oder ```delmax``` aufgerufen wird. In diesem Fall wird das grösste Element gesucht und mit dem letzten Element vertauscht. Damit kann es einfach zurückgegeben werden.

In [1]:
class MaxPQUnorderedArray:
    
    def __init__(self):
        self.data = []
    
    def insert(self, k):
        self.data.append(k)
    
    def max(self):
        self._largestToEnd()
        return self.data[-1] #[-1] greift auf letztes Element zu
            
    def delMax(self):
        self._largestToEnd()
        return self.data.pop()
    
    def _largestToEnd(self):
        if len(self.data) == 0:
            return
        
        maxElem = self.data[0]            
        maxIndex = 0
        for (i, d) in enumerate(self.data):
            if (maxElem < d):
                maxElem = self.data[i]
                maxIndex = i
        (self.data[maxIndex], self.data[-1]) = (self.data[-1], self.data[maxIndex])
        
    
        
    def isEmpty(self):
        return len(self.data) == 0
    
    def size(self):
        return len(self.data)

#### Implementation mit sortiertem Array

In dieser zweiten Implementation, passiert die Hauptarbeit in der Methode ```insert```. Unsere Implementation stellt sicher, dass das neue Element jeweils an die richtige Position im Array eingefügt wird. Im Vergleich zur vorherigen Methode, sind dafür die Methoden ```max``` und ```delMax``` sehr effizient.

In [2]:
class MaxPQOrderedArray:
    
    def __init__(self):
        self.data = []
    
    def insert(self, k):
        
        # Suche im sortierten Array data die Position vom neuen Element
        idx = 0
        while (idx < len(self.data) and self.data[idx] < k):            
            idx += 1
        self.data.insert(idx, k)        
        
    def max(self):
        if self.isEmpty():
            return None
        else:
            return self.data[-1]
            
    def delMax(self):
        return self.data.pop()
        
        
    def isEmpty(self):
        return len(self.data) == 0
    
    def size(self):
        return len(self.data)

#### Komplexität

Nun schauen wir uns die Laufzeit der beiden Implementationen an. Wir nutzen dafür wieder das Python Modul ```timeit``` welches wir schon früher kennengelernt haben. 

In [3]:
import timeit
import random

Die folgenden beiden Hilfsfunktionen fügen jeweils $N$-Elemente hinzu oder entfernen diese aus der Queue.

In [4]:
def insertNElements(pq, N):
    for i in range(0, N):
        pq.insert(random.randint(0, N))


In [5]:
def removeLargestNElements(pq, N):
    for i in range(0, N):
        pq.delMax()

Wir können die Tests nun ausführen. 

In [6]:
orderedPQ = MaxPQOrderedArray()
unorderedPQ = MaxPQUnorderedArray()

print("insert ordered ", timeit.timeit(lambda: insertNElements(orderedPQ, 10000), number = 1))
print("insert unordered", timeit.timeit(lambda: insertNElements(unorderedPQ, 10000), number = 1))

# ACHTUNG: Wir nutzen hier aus, dass orderedPQ und unorderedPQ bereits wegen dem vorigen Experiment gefüllt sind
print("removeLargest ordered", timeit.timeit(lambda: removeLargestNElements(orderedPQ, 10000), number = 1))
print("removeLargest unordered", timeit.timeit(lambda: removeLargestNElements(unorderedPQ, 10000), number = 1))

insert ordered  8.7629225
insert unordered 0.024069000000000784
removeLargest ordered 0.0033111000000012325
removeLargest unordered 4.5518407


Wie erwartet ist das Einfügen sehr viel teurer, wenn wir für die Implementation ein geordnetes Array nutzen. Dafür wird das Entfernen des grössten Elements entsprechend effizienter. 

### Beispielanwendung von Priority-Queues

Eine typische Beispielanwendung ist, dass man aus einem sehr grossen Datenstrom die extremsten Elemente extrahieren möchte. 

In unserem Beispiel besteht der Datenstrom aus normalverteilten Zufallswerten. Wir schreiben einen Client, der die $M$ kleinsten Werte, die generiert wurden, ausgibt. 

Die folgende Funktion generiert einen Stream von $N$ normalverteilen Zufallszahlen. Das Python Keyword yield stellt hier sicher, dass die Zahlen jeweils einzeln generiert werden, so dass wir nicht eine grosse Liste von Zahlen speichern müssen. 

In [7]:
import random

def numberGen(N):
    num = 0
    while num < N:
        yield random.gauss(0, 1)
        num += 1

In [8]:
n = numberGen(100)
for i in range(0, 1):
    print(next(n))

0.29263556528650453


Da wir hier nur eine MaxPQ zur Verfügung haben, ist es etwas einfacher die kleinsten Elemente zu suchen. Wir speichern jedes Element in der PriorityQueue, lassen aber nur jeweils die $M$ grössten Elemente in der Queue. Die anderen werden sofort wieder entfernt.

In [9]:
def printSmallestNumbers(M, N):
    pq = MaxPQUnorderedArray()
    for number in numberGen(N):

        pq.insert(number) 
        if pq.size() > M:
            pq.delMax() 
        
    while not pq.isEmpty():
        print(pq.delMax())


In unserer Anwendung können wir nun mit diesem Algorithmus ermitteln, wie extrem die Werte, die zufällig von einer Standard-Normalverteilung gezogen werden, eigentlich werden können. 

In [10]:
printSmallestNumbers(1000, 1000000)

-3.0967224211192965
-3.0968948285895563
-3.0969898864591134
-3.0972845269497435
-3.0974401376064424
-3.097553061254444
-3.097631801655745
-3.0990387252568317
-3.099567182081363
-3.0999346030516794
-3.0999483702097104
-3.1000088114015356
-3.100240834842832
-3.1010677620848504
-3.1017194916991646
-3.1018383814329646
-3.1021557748959254
-3.1024701144835483
-3.1026832876418555
-3.1026871454426352
-3.10299416683825
-3.103152534594986
-3.1033997976076466
-3.1034509865565973
-3.103541656258665
-3.104151622861328
-3.104444099172476
-3.1048674143450974
-3.1052582885412146
-3.1053491545659093
-3.1056249441080586
-3.1059442919782376
-3.1061457261337413
-3.1062558801346136
-3.1069143736151106
-3.1072482663627334
-3.1073491608639685
-3.1073554232976037
-3.107690270364301
-3.1084961847223576
-3.1085895468987017
-3.109419059556538
-3.110421715650572
-3.110442645521659
-3.110817627428167
-3.111124125843531
-3.1112324612998328
-3.1112860163334886
-3.1116769980694423
-3.1121295975595142
-3.1130610188505

-3.3094138754797697
-3.3096821235725873
-3.310555367442231
-3.312170759281685
-3.3138579493215565
-3.314748644972639
-3.31716071064719
-3.3173154059942616
-3.3173614959005957
-3.3179077535789934
-3.3194536887327537
-3.3197004792478704
-3.3201508237976514
-3.3205565259585303
-3.3210075040903053
-3.321363054274972
-3.3221231302045666
-3.323064240047081
-3.326052658292955
-3.326449772287413
-3.32684454460568
-3.3275182677828554
-3.3276266650799022
-3.3281473820442122
-3.328606315711914
-3.328658928677384
-3.329161866469132
-3.3299329695616793
-3.330480743769451
-3.3311128437527042
-3.333757478147544
-3.3344455265615913
-3.335122737431693
-3.3356339952288514
-3.3358493686211617
-3.336061576018787
-3.3383036936355537
-3.3385682663705545
-3.3396079049099026
-3.3399273677745223
-3.340905785208091
-3.341142241813669
-3.3422763953171373
-3.3425682618762393
-3.343255249008706
-3.3435153058326175
-3.34387560376189
-3.343950130666276
-3.3441504865836267
-3.34469184131739
-3.3455350634190935
-3.345

Wir sehen, dass auch wenn wir 1 Million zufällige Elemente ziehen, kaum eines extremer als -5 ist. 

*Anmerkung für Statistik-Interessierte:  Unter der Normalverteilung sind, wie wir sehen, extreme Werte sehr unwahrscheinlich. Deshalb ist die Normalverteilung kein gutes Modell für Prozesse, bei denen auch ab und zu mal ein grösserer Wert vorkommt.*

### Sortieren mithilfe einer Priorityqueue

Wir können den Datentyp Priorityqueue auch nutzen um Daten zu sortieren. Dazu speichern wir die Daten einfach alle in der Priorityqueue und entfernen diese der Grösse nach.

* Was ist die Laufzeit mit unserer naiven Implementation einer PQ?
* Ist dies ein in-place Verfahren?

In [15]:
testdata = [1,5, 2, 8 , 11]
pq = MaxPQOrderedArray()
for t in testdata:
    pq.insert(t)

sortedArray = []
while not pq.isEmpty():
    sortedArray.append(pq.delMax())

In [16]:
print(sortedArray)

[11, 8, 5, 2, 1]
